<a href="https://colab.research.google.com/github/krisibraynova01/Thesis_TechnicalUnifersityofSofia_Master/blob/main/Predictions_delivery_status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, mean_squared_error, mean_absolute_error, r2_score
import scipy.stats as stats
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor

In [27]:
df = pd.read_csv("SupplyChain_preprocessed.csv")

<ipython-input-27-ca8d0b4c8251>:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("SupplyChain_preprocessed.csv")


In [28]:
prediction_data=df[['Order Region','Shipping Mode','Delivery Status']]

In [29]:
prediction_data.columns=['Order_Region','Shipping_Mode','Delivery_Status']

In [30]:
prediction_data=prediction_data[prediction_data.Delivery_Status!='Shipping canceled']
prediction_data=prediction_data[prediction_data.Delivery_Status!='Advance shipping']
prediction_data.head()

,Order_Region,Shipping_Mode,Delivery_Status
1,South Asia,Standard Class,Late delivery
2,South Asia,Standard Class,Shipping on time
6,Eastern Asia,First Class,Late delivery
7,Eastern Asia,First Class,Late delivery
8,Eastern Asia,Second Class,Late delivery


In [31]:
prediction_data=pd.get_dummies(prediction_data, drop_first=True)
prediction_data.head()

,Order_Region_Caribbean,Order_Region_Central Africa,Order_Region_Central America,Order_Region_Central Asia,Order_Region_East Africa,Order_Region_East of USA,Order_Region_Eastern Asia,Order_Region_Eastern Europe,Order_Region_North Africa,Order_Region_Northern Europe,...,Order_Region_US Center,Order_Region_West Africa,Order_Region_West Asia,Order_Region_West of USA,Order_Region_Western Europe,Shipping_Mode_Same Day,Shipping_Mode_Second Class,Shipping_Mode_Standard Class,Delivery_Status_Late delivery,Delivery_Status_Shipping on time
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,True
6,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
7,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,True,False
8,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,True,False


In [32]:
X_train, X_test, Y_train, Y_test=train_test_split(prediction_data.drop('Delivery_Status_Shipping on time',axis=1),prediction_data['Delivery_Status_Shipping on time'])

In [52]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:
LogReg=LogisticRegression()
LogReg.fit(X_train_scaled, Y_train)

LogisticRegression()

In [44]:
LogReg.score(X_test_scaled, Y_test)

0.9999498822232246

In [45]:
Y_pred=LogReg.predict(X_test)
print (classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

       False       0.76      1.00      0.86     15182
        True       0.00      0.00      0.00      4771

    accuracy                           0.76     19953
   macro avg       0.38      0.50      0.43     19953
weighted avg       0.58      0.76      0.66     19953



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze

In [46]:
classifier = RandomForestClassifier()
classifier.fit(X_train_scaled,Y_train)

RandomForestClassifier()

In [47]:
classifier.score(X_test_scaled,Y_test)

0.9999498822232246

In [51]:
Y_pred_RF=classifier.predict(X_test)
print (classification_report(Y_test,Y_pred_RF))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


              precision    recall  f1-score   support

       False       0.76      1.00      0.86     15182
        True       0.00      0.00      0.00      4771

    accuracy                           0.76     19953
   macro avg       0.38      0.50      0.43     19953
weighted avg       0.58      0.76      0.66     19953



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
